## 계층적 멀티 에이전트 팀(Hierarchial Agent Team)

시스템에 더 많은 에이전트를 추가하면 슈퍼바이저가 모든 에이전트를 관리하기 너무 어려워질 수 있습니다. 슈퍼바이저가 다음에 호출할 에이전트에 대해 잘못된 결정을 내리기 시작하거나, 단일 슈퍼바이저가 추적하기에는 컨텍스트가 너무 복잡해질 수 있습니다. 즉, 애초에 멀티 에이전트 아키텍처를 채택하게 된 것과 동일한 문제가 발생하게 됩니다.

이를 해결하기 위해 시스템을 **계층적으로** 설계할 수 있습니다. 예를 들어, 개별 슈퍼바이저가 관리하는 전문화된 에이전트 팀을 별도로 만들고, 최상위 슈퍼바이저가 팀을 관리하도록 할 수 있습니다.

- 참고 자료: https://wikidocs.net/270691
- 관련 논문: https://arxiv.org/abs/2308.08155

## 환경 설정

In [1]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv("../.env", override=True)


def _set_env(var: str):
    env_value = os.environ.get(var)
    if not env_value:
        env_value = getpass.getpass(f"{var}: ")

    os.environ[var] = env_value


_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"
_set_env("OPENAI_API_KEY")

## ResearchTeam

In [3]:
from langchain_tavily import TavilySearch

# 웹 검색 도구 정의
web_search_tool = TavilySearch(max_results=5)

In [ ]:
# 웹 페이지에서 세부 정보를 스크래핑하기 위한 도구 정의
from typing import List
from langchain_core.tools import tool
from langchain_community.document_loaders import WebBaseLoader


@tool
def scrape_webpages(urls: List[str]):
    """제공된 웹 페이지에서 상세 정보를 추출하기 위해 사용하십시오."""
    loader = WebBaseLoader(
        web_paths=urls,
        header_template={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/110",
        },
    )
    docs = loader.load()

    return "\n".join(
        [
            f'<Document title="{doc.metadata.get("title", "")}" url="{doc.metadata.get("source", "")}">{doc.page_content}</Document>'
            for doc in docs
        ]
    )
    return docs


In [13]:
scrape_webpages.invoke(
    input={
        "urls": [
            "https://langchain-ai.github.io/langgraph/concepts/multi_agent/#handoffs-as-tools"
        ]
    }
)

[Document(metadata={'source': 'https://langchain-ai.github.io/langgraph/concepts/multi_agent/#handoffs-as-tools', 'title': 'Overview', 'description': 'Build reliable, stateful AI systems, without giving up control', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\nOverview\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n          Skip to content\n        \n\n\n\n\n\n\n\n            \n            \nThese docs will be deprecated and removed with the release of LangGraph v1.0 in October 2025. Visit the v1.0 alpha docs\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            LangGraph\n          \n\n\n\n            \n              Overview\n            \n          \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Initializing search\n          \n\n\n\n\n\n\n\n\n\n\n\n\n    GitHub\n  \n\n\n\n\n\n\n\n\n\n\n          \n  \n  \n    \n  \n  Get started\n\n        \n\n\n\n          \n  \n  \n    \n  \n  Guides\n\n        \n\n\n\n          \n  \n  